In [ ]:
import torch
import os
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("joeddav/distilbert-base-uncased-go-emotions-student")
model = AutoModelForSequenceClassification.from_pretrained("joeddav/distilbert-base-uncased-go-emotions-student")
genres = list(model.config.id2label.values())

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def inferencing(text, threshold = 0.8, model=model, tokenizer=tokenizer):
    encoding = tokenizer.encode_plus (
        text,
        return_tensors = "pt",
        max_length = 512,
        padding = "max_length",
        truncation = True
    )
    
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)
    outputs = torch.sigmoid(outputs.logits).detach().cpu().numpy().tolist()[0]
    outputs = np.array(outputs) >= threshold
    outputs = np.where(outputs == True)[0]
    outputs = [genres[i] for i in outputs]
    return outputs

In [5]:
inferencing("I feel lucky to be here.")

['gratitude', 'optimism', 'relief']